In [50]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from selenium import webdriver
from urllib.request import urlopen
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [52]:
path = '/Users/yichuanhong/Documents/Python Project/Vietnam Avian Flu Outbreak Research/NLP/chromedriver'
url = 'https://tuoitre.vn/tim-kiem.htm?keywords=c%C3%BAm%20gia%20c%E1%BA%A7m'
root = 'https://tuoitre.vn'
df = pd.DataFrame(columns = ['Title', 'URL', 'Date', 'Content'])
driver = webdriver.Chrome(path)
driver.get(url)

In [53]:
while True:
    try:
        loadMoreButton = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[3]/div/div[1]/div/div/div/a')
        loadMoreButton.click()
        time.sleep(2)
    except:
        print('Reached bottom of page')
        break

Reached bottom of page


In [54]:
news = driver.find_elements_by_css_selector('div.name-news')

In [55]:
for item in news:
    cur = item.find_element_by_tag_name('a')
    title = cur.get_attribute('title')
    sub_url = cur.get_attribute('href')
    if sub_url.startswith('https'):
        url = sub_url
        response = requests.get(url).text
    else:
        url = root + sub_url
        response = requests.get(url).text
    soup = BeautifulSoup(response, 'lxml')
    date = soup.find('div', class_ = 'date-time').text
    date = datetime.strptime(date.split(' ')[0], '%d/%m/%Y')
    content = soup.find('div', class_ = 'content fck').text
    new_row = {'Title': title, 'URL': url, 'Date': date, 'Content': content}
    df = df.append(new_row, ignore_index = True)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=92.0.4515.107)


In [56]:
df

,Title,URL,Date,Content
0,Quảng Ninh phát hiện chủng cúm gia cầm lần đầu...,https://tuoitre.vn/quang-ninh-phat-hien-chung-...,2021-07-02,"\nNgày 2-7, Chi cục Chăn nuôi và thú y tỉnh Qu..."
1,Trung Quốc ghi nhận ca nhiễm virus cúm gia cầm...,https://tuoitre.vn/trung-quoc-ghi-nhan-ca-nhie...,2021-06-01,\nTiêm ngừa cúm cho gia cầm ở Trung Quốc - Ảnh...
2,"Mới 5 tuần đã có 40 ổ dịch cúm gia cầm, nguy c...",https://tuoitre.vn/moi-5-tuan-da-co-40-o-dich-...,2021-02-08,\nNguy cơ lây nhiễm cúm gia cầm sang người - Ả...
3,"Lo dịch bệnh tấn công gia súc, gia cầm",https://tuoitre.vn/lo-dich-benh-tan-cong-gia-s...,2020-12-25,"\nMột ổ dịch cúm gia cầm tại huyện Cần Đước, L..."
4,"Cúm gia cầm đe dọa Nhật, Hàn bị dạng cúm độc l...",https://tuoitre.vn/cum-gia-cam-de-doa-nhat-han...,2020-12-13,\nLực lượng chức năng chuẩn bị xử lý cúm gia c...
...,...,...,...,...
556,Trung Quốc: Bộ Y tế thừa nhận khả năng cúm gia...,https://tuoitre.vn/trung-quoc-bo-y-te-thua-nha...,2008-01-11,"\nÔng Lục, 52 tuổi, bị nhiễm bệnh chỉ vài ngày..."
557,Cúm gia cầm lây từ người sang người tại Trung ...,https://tuoitre.vn/cum-gia-cam-lay-tu-nguoi-sa...,2008-01-10,\nPhóng toMột trang trại gia cầm tại Trung Quố...
558,Phát hiện cách virus cúm gia cầm lây truyền sa...,https://tuoitre.vn/phat-hien-cach-virus-cum-gi...,2008-01-09,\nPhóng toỞ một chợ gà -Ảnh: A.F.PTT - Ngày 6-...
559,Dịch cúm gia cầm có nguy cơ tái phát,https://tuoitre.vn/dich-cum-gia-cam-co-nguy-co...,2008-01-06,\nHiện vẫn còn tỉnh Trà Vinh và Thái Nguyên có...


In [57]:
df.to_csv('data/tuoitre_articles_vietnamese.csv', index = False)

In [49]:
driver.quit()